## Задачи
 1. Импортируйте библиотеку pandas как pd. Загрузите датасет bookings.csv с разделителем ;. Проверьте размер таблицы, типы переменных, а затем выведите первые 7 строк, чтобы посмотреть на данные. 
 2. Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.
 3. Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.
 4. На сколько ночей в среднем бронируют отели разных типов?
 5. Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?
 6. Проанализируйте даты запланированного прибытия. – На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?– Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов
 7. Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?
 8. Создайте колонку total_kids, объединив children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?
 9. Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.

In [9]:
# first task
path_to_file = '2_bookings.csv'
import pandas as pd 
bookings = pd.read_csv(path_to_file, sep = ';')
bookings_head = bookings.head(7)

In [10]:
# second task
bookings.shape

(119390, 21)

In [11]:
# third task
bookings.info()
# object + int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  object 
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  object 
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  object 
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     11

In [12]:
# fourth task
bookings.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

In [13]:
# fifth task
(
    bookings
    .query('is_canceled == 0')
    .country
    .value_counts()
    .sort_values(ascending=False)
    .head(5)
)           
               

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

In [14]:
# sixth task
(
    bookings
    .groupby('hotel', as_index=False)
    .agg({'stays_total_nights' : 'mean'})
    .round(decimals=2)
)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [15]:
# seventh task
(bookings.assigned_room_type != bookings.reserved_room_type).sum()

14917

In [16]:
# eigth task
(
    bookings
    .groupby(['arrival_date_year', 'arrival_date_month'])
    .agg({'arrival_full_date' : 'count'})
    .groupby('arrival_date_year')
    .idxmax()
)

,arrival_full_date
arrival_date_year,
2015,"(2015, September)"
2016,"(2016, October)"
2017,"(2017, May)"


In [17]:
# nineth task
(
    bookings
    .query("hotel == 'City Hotel'")
    .groupby(['arrival_date_year', 'arrival_date_month'])
    .agg({'is_canceled' : 'sum'})
    .groupby('arrival_date_year')
    .idxmax()
)

,is_canceled
arrival_date_year,
2015,"(2015, September)"
2016,"(2016, October)"
2017,"(2017, May)"


In [18]:
# tenth task
bookings[['adults', 'children', 'babies']].mean()

adults      1.856403
children    0.103890
babies      0.007949
dtype: float64

In [19]:
# eleventh task
bookings['total_kids'] = bookings['children'] + bookings['babies']
(
    bookings
    .groupby('hotel')
    .total_kids
    .mean()
    .round(decimals=2)
)


hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

In [22]:
# twelfth task
bookings['has_kids'] = bookings.total_kids > 0
temp = (
    bookings[['has_kids', 'is_canceled']]
    .groupby(['has_kids', 'is_canceled'], as_index=False)
    .value_counts()
    .pivot(index = 'has_kids', columns = 'is_canceled', values = 'count')
    .reset_index()
)

temp['total'] = temp[0] + temp[1]
temp['churn_percent'] = (temp[1] / temp['total'])*100
temp.round(2)

is_canceled,has_kids,0,1,total,churn_percent
0,False,69093,40965,110058,37.22
1,True,6073,3259,9332,34.92
